# Banking Customer Segmentation

First we start off by loading the necessary libraries.  
We are also going to use a dotenv file to store our variables such as path to the csv file.


In [85]:
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

load_dotenv()

True

Load the data as a pandas dataframe 


In [86]:
data_path = os.getenv('DATA_PATH')

bank_data = pd.read_csv(data_path)

bank_data

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5
...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,8/4/90,M,NEW DELHI,7635.19,18/9/16,184824,799.0
1048563,T1048564,C6459278,20/2/92,M,NASHIK,27311.42,18/9/16,183734,460.0
1048564,T1048565,C6412354,18/5/89,M,HYDERABAD,221757.06,18/9/16,183313,770.0
1048565,T1048566,C6420483,30/8/78,M,VISAKHAPATNAM,10117.87,18/9/16,184706,1000.0


In [87]:
bank_data.count()

TransactionID              1048567
CustomerID                 1048567
CustomerDOB                1045170
CustGender                 1047467
CustLocation               1048416
CustAccountBalance         1046198
TransactionDate            1048567
TransactionTime            1048567
TransactionAmount (INR)    1048567
dtype: int64

In [88]:
bank_data.isna().sum()

TransactionID                 0
CustomerID                    0
CustomerDOB                3397
CustGender                 1100
CustLocation                151
CustAccountBalance         2369
TransactionDate               0
TransactionTime               0
TransactionAmount (INR)       0
dtype: int64

In [89]:
bank_data.dropna(inplace=True)

In [90]:
bank_data.count()

TransactionID              1041614
CustomerID                 1041614
CustomerDOB                1041614
CustGender                 1041614
CustLocation               1041614
CustAccountBalance         1041614
TransactionDate            1041614
TransactionTime            1041614
TransactionAmount (INR)    1041614
dtype: int64

In [91]:
print(bank_data['CustGender'].unique())
print('M:', len(bank_data[bank_data['CustGender'] == 'M']))
print('F:', len(bank_data[bank_data['CustGender'] == 'F']))
print('T:', len(bank_data[bank_data['CustGender'] == 'T']))

['F' 'M' 'T']
M: 760978
F: 280635
T: 1


From the above, we see there is only one gender classified as T which doesn't add significant value to the data so we can remove it 

In [92]:
bank_data.drop(bank_data[bank_data['CustGender'] == 'T'].index, inplace=True)
print('M:', len(bank_data[bank_data['CustGender'] == 'M']))
print('F:', len(bank_data[bank_data['CustGender'] == 'F']))
print('T:', len(bank_data[bank_data['CustGender'] == 'T']))


M: 760978
F: 280635
T: 0


We observed some Customer DOB are of the year 1800 which doesn't make logical sense so we will remove those 

In [93]:
# duplicate_index = duplicates[duplicates > 1].index
# print(duplicate_index)
# filtered_duplicates = bank_data[bank_data['CustomerID'].isin(duplicate_index)]
# filtered_duplicates
# bank_data[bank_data['CustomerID'] == 'C5533885']
print(bank_data['CustomerDOB'].value_counts())

CustomerDOB
1/1/1800    56291
1/1/89        809
1/1/90        784
6/8/91        698
1/1/91        665
            ...  
2/12/51         1
20/3/52         1
26/9/47         1
4/10/41         1
24/10/44        1
Name: count, Length: 17233, dtype: int64


In [94]:
bank_data.drop(bank_data[bank_data['CustomerDOB'] == '1/1/1800'].index, inplace=True)
print(bank_data['CustomerDOB'].value_counts())


CustomerDOB
1/1/89      809
1/1/90      784
6/8/91      698
1/1/91      665
1/1/92      631
           ... 
23/2/05       1
28/11/42      1
23/9/49       1
14/3/40       1
24/10/44      1
Name: count, Length: 17232, dtype: int64


In [113]:
bank_data["CustomerDOB"] = pd.to_datetime(bank_data["CustomerDOB"])
bank_data["TransactionDate"] = pd.to_datetime(bank_data["TransactionDate"])
bank_data.loc[bank_data.CustomerDOB.dt.year >= bank_data.TransactionDate.dt.year, 'CustomerDOB']  = bank_data.loc[bank_data.CustomerDOB.dt.year >= bank_data.TransactionDate.dt.year, 'CustomerDOB'] - pd.DateOffset(years=100)
bank_data

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,143207,25.0
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,141858,27999.0
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,142712,459.0
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,142714,2060.0
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,181156,1762.5
...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,1990-08-04,M,NEW DELHI,7635.19,2016-09-18,184824,799.0
1048563,T1048564,C6459278,1992-02-20,M,NASHIK,27311.42,2016-09-18,183734,460.0
1048564,T1048565,C6412354,1989-05-18,M,HYDERABAD,221757.06,2016-09-18,183313,770.0
1048565,T1048566,C6420483,1978-08-30,M,VISAKHAPATNAM,10117.87,2016-09-18,184706,1000.0


Add a customer age column

In [116]:
bank_data["CustomerAge"] = ((bank_data["TransactionDate"].max() - bank_data["CustomerDOB"]).dt.days/365).round(0)
bank_data

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustomerAge
0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,143207,25.0,22.0
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,141858,27999.0,60.0
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,142712,459.0,20.0
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,142714,2060.0,43.0
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,181156,1762.5,29.0
...,...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,1990-08-04,M,NEW DELHI,7635.19,2016-09-18,184824,799.0,26.0
1048563,T1048564,C6459278,1992-02-20,M,NASHIK,27311.42,2016-09-18,183734,460.0,25.0
1048564,T1048565,C6412354,1989-05-18,M,HYDERABAD,221757.06,2016-09-18,183313,770.0,28.0
1048565,T1048566,C6420483,1978-08-30,M,VISAKHAPATNAM,10117.87,2016-09-18,184706,1000.0,38.0


We also need to deal with customer ID duplicates  
- Need to choose either to delete the customers or 
- Find 


In [182]:
duplicates = bank_data.pivot_table(index=['CustomerID'], aggfunc='size')
duplicate_customer_id = duplicates[duplicates > 1].sort_values(ascending=False)
duplicate_customer_id

CustomerID
C1736254    6
C6222360    6
C6735477    6
C1026833    6
C6624352    6
           ..
C3840149    2
C3840145    2
C3840134    2
C3840075    2
C9096252    2
Length: 128896, dtype: int64

In [185]:
duplicate_customers_df = bank_data[bank_data['CustomerID'].isin(duplicate_customer_id.index)]
duplicate_customers_df

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustomerAge
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,142714,2060.0,43.0
6,T7,C7126560,1992-01-26,F,MUMBAI,973.46,2016-02-08,173806,566.0,25.0
10,T11,C5430833,1982-07-22,M,MOHALI,48429.49,2016-02-08,204133,259.0,34.0
27,T28,C2416848,1993-01-04,M,CHENNAI,20986.66,2016-03-08,160544,2235.0,24.0
41,T42,C7923455,1992-06-10,M,ANDHERI (EAST) MUMBAI,5810.82,2016-03-08,221111,12.0,25.0
...,...,...,...,...,...,...,...,...,...,...
1048549,T1048550,C4819452,1990-10-30,F,THANE THANE,1600.14,2016-09-18,184226,1094.0,26.0
1048555,T1048556,C2122724,1978-03-20,M,FARIDABAD,436598.03,2016-09-18,185337,5259.0,39.0
1048556,T1048557,C3414578,1990-10-09,F,GURGAON,27592.70,2016-09-18,184932,2000.0,26.0
1048564,T1048565,C6412354,1989-05-18,M,HYDERABAD,221757.06,2016-09-18,183313,770.0,28.0


In [209]:
grouped = duplicate_customers_df.groupby(['CustomerID', 'CustomerDOB']).size()
same_dob = grouped[grouped > 1]
same_dob = same_dob.sort_values(ascending=False)
dob_counts_df = same_dob.reset_index(name='Count')
dob_counts_df


0       C1224627
1       C3741055
2       C2140114
3       C1422287
4       C6347839
          ...   
1210    C3619025
1211    C3616833
1212    C3611555
1213    C3610683
1214    C9083021
Name: CustomerID, Length: 1215, dtype: object

In [212]:
grouped_duplicate_customers_df = duplicate_customers_df[duplicate_customers_df["CustomerID"].isin(dob_counts_df.CustomerID)]
grouped_duplicate_customers_df

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustomerAge
259,T260,C3531473,1989-03-12,M,KOLKATA,13592.65,2016-10-21,154628,1128.37,28.0
269,T270,C2526046,1988-07-01,F,KOLKATA,31369.53,2016-10-21,151510,1735.00,28.0
507,T508,C3830864,1987-05-25,M,IDUKKI,134.18,2016-10-21,212736,240.00,30.0
1172,T1173,C4740088,1976-03-17,M,GURGAON,20278.39,2016-10-21,210249,55.00,41.0
1232,T1233,C5141632,1991-12-24,F,DELHI,118.51,2016-10-21,210237,280.00,25.0
...,...,...,...,...,...,...,...,...,...,...
1046591,T1046592,C7114861,1993-11-15,M,LUDHIANA,47.17,2016-09-18,72421,30.00,23.0
1046899,T1046900,C2173491,1972-02-26,M,VADODARA,95445.47,2016-09-18,184956,999.00,45.0
1047401,T1047402,C8248728,1989-03-20,F,NOIDA,149.47,2016-09-18,175810,1650.00,28.0
1048043,T1048044,C6115521,1994-03-07,F,HYDERABAD,14001.22,2016-09-18,82030,85.00,23.0


In [233]:
bank_data.loc[(bank_data.CustomerAge < 18) | (bank_data.CustomerAge > 90) , "CustomerAge"] # do we drop these ages or not? 


294        17.0
390        96.0
485        12.0
734        17.0
1328       96.0
           ... 
1045119    14.0
1045629    11.0
1046347    17.0
1046676    15.0
1047129     1.0
Name: CustomerAge, Length: 1221, dtype: float64